# Trabajando con graficas

## Introduccion ##
A continuación se van a replicar algunos de los ejercicios del libro throbleshooting wireshark con el fin de entender mejor el uso de la herramienta y de sacar graficas para armar discusiones con el fin de filtrar las mas importantes para analisis posteriores.

### Escenario 1 ### 

**tr-twohosts.pcapng**: Contiene el trafico a y desde dos hosts. Un host realiza descarfas a velocidades lentas desde un site mientras que el otro no.

**Pasos**:
1. Abrir archivo.
2. Determinar IPs: **con Statistics > Conversations** de acuerdo a las conversaciones (flujos) arrojadas determinar las IPs.
3. Realizar la grafica: **Statistics > IO Graph** para graficar y cacharrear. La grafica negra no tiene filtros.
4. Emplear los filtros:
   1. Grafica roja: ``ip.addr==192.168.1.72``
   2. Grafica azul: ``ip.addr==192.168.1.119``

![grafica1](two_hosts.png)

Se puede ver un mejor throughput entre los dos proceso de descarga La descarga hecha por 192.168.1.72 se completa mas rapidamente que la de 192.168.1.119.

### Escenario 2  - Graficando todo el trafico para una unica aplicacion. ###

**tr-ftphttp.pcapng**: Contiene el trafico FTP (incluyendo comandos FTP ytransferencia de datos) y HTTP (basado en los numeros de puerto en uso). 

**Pasos**
1. Ver conversaciones: **Statistics > Conversations > TCP** y deseleccione el chechbox **Name resulution**. Observe los puertos usados:
   1. Trafico HTTP: 80.
   1. Trafico FTP: 21.
   1. Canal de datos FTP: 22487. 
   
2. Filtros empleados:
   1. Grafica roja: ``tcp.port==80`` (line style)
   2. Grafica azul: ``tcp.port==21 or tcp.port==22487`` (Impulse style)

![grafica2](ftp_http1.png)

**Nota**: Las tasas de trafico pico hacen dificil ver si hay una baja tasa de trafico en puntos del archivo de trazas. Aplicar escalas logaritmicas ayuda a visualizar bajas escalas logaritmicas.


![grafica3](ftp_http2_log.png)


**Tip**: Cuando grafique dos numeros disparados, fije la escala del eje Y a escala logaritmica para ver los numeros mas pequeños a lo largo  de los numeros mas grandes.



**¿Que interpretaciones podemos sacar?**
1. Tamaño.
2. Por cuanto tiempo.
3. etc.

### Escenario 3 -  Detect Consistently Low Throughput due to Low Packet Sizes ###

**Contexto**:
Los Low packet sizes pueden ser causados por una aplicacion que intencionalmente dese transferir cantidades mas pequeñas de datos. Los Low packet sizes tambien pueden ser un indicatos de un bajo Maximum Segment Size (MSS) asignado. Un bajo MSS puede ser debido a una misconfiguration en el cliente o aun en una funcionalidad adicional.

**tr-throughput.pcapng**: esta traza consiste de una conexion HTTPs entre dos host. Como no es necesario desencriptar el codigo, solo se puede hanalizar hasta el punto de la capa TCP.

**Pasos**:
1. Muevase por la traza y observe los tamaños de la columna **length** para cada paquete. Se notara que hay muchos paquetes de pequeña longitud
2. En los paquetes 1 y 2 (donde se hace parte del handshake) vaya a la parte asociada al componente TCP y busque el valor que se asigno en MSS (en el campo options de la cabecera TCP). Para el caso 1460 bytes. **COnclusion**: El problema no es po un bajo valor de MSS.
3. Realizar la grafica: **Statistics > IO Graph** (Colocar la escala del eje Y a 2000)

![caso3_grafica1](caso3_gr1.png)

La transferencia de datos tiene picos alrededor de los 1900 bps.

4. Obtener el resumen: **Statistics > Summary**

![caso3_grafica2](summary.png)

Aqui el **average packet size in the trace file** es aproximadamente 145 bytes (pero estos incluyen los paquetes con ACK).

![caso3_grafica3](summary2.png)

En la figura anterior se aplico un filtro para **average data packet size** y es ``tcp.len > 0`` con esto se ven los paquetes que contienen solo datos. En el caso el resultado es 213 bytes.

**Conclusiones**:
Why are smaller packets being used? We identified that it isn't a limitation defined by the TCP
connection. This is likely an application setting. We must look at the application to determine if this is
intentional or a configuration problem.

### Escenario 4 - Identify Queuing Delays along a Path ###

**Contexto**:
Interconectar dispositivos pueden inyectar retardos al queuing (holding the packets temporarily before
forwarding them) a lo largo de la ruta.

Consider using a traffic generator to detect queuing along a path. A tool such as iPerf/jPerf can be
used to transmit traffic at a steady rate. We need that steady rate of transmission to identify distinct
queuing patterns when you capture traffic on the other side of a queuing device. In the next lab, we will look at the IO Graph indication of traffic that has been queued along the path.

**Objetivo**: Identify the Queued Traffic Pattern in an IO Graph

**tr-queuing.pcapng**: Contiene un video multicas que transmite paquetes a una direccion multicas 239.255.0.1

**Pasos**:

1. Realizar la grafica: **Statistics > IO Graph** 